In [123]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [124]:
df=pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [125]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)

In [126]:
x_train_scaled=x_train.copy()
x_train_scaled['age']=x_train_scaled['age']/100
x_test_scaled=x_test.copy()
x_test_scaled['age']=x_test_scaled['age']/100

In [ ]:
model=keras.Sequential([
    keras.layers.Dense(1,activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train_scaled,y_train,epochs=367)

Epoch 1/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.7113
Epoch 2/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5000 - loss: 0.7110
Epoch 3/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5000 - loss: 0.7106
Epoch 4/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5000 - loss: 0.7102
Epoch 5/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.5000 - loss: 0.7098
Epoch 6/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.5000 - loss: 0.7094
Epoch 7/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5000 - loss: 0.7091
Epoch 8/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5000 - loss: 0.7087
Epoch 9/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5000 - loss: 0.7083
Epoch 10/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.5000 - loss: 0.7079
Epoch 11/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5000 - loss: 0.7076
Epoch 12/367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5000 - loss

In [128]:
model.evaluate(x_test_scaled,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.6667 - loss: 0.6210


[0.621041476726532, 0.6666666865348816]

In [129]:
model.predict(x_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


array([[0.66075516],
       [0.60847205],
       [0.451497  ],
       [0.62685424],
       [0.66423655],
       [0.68474233]], dtype=float32)

In [130]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [131]:
coef,intersept=model.get_weights()
coef,intersept

(array([[0.77850705],
        [0.6977987 ]], dtype=float32),
 array([-0.39703578], dtype=float32))

In [132]:
def prediction_function(x1,x2):
    weighted_sum=coef[0]*x1+coef[1]*x2+intersept
    return sigmoid(weighted_sum)
prediction_function(.47,1)

array([0.66075516], dtype=float32)

In [133]:
def sigmoid(z):
    s=1/(1+np.exp(-z))
    return s

In [134]:
def log_loss(y_truth,y_pred):
    epsilon=1e-15
    y_pred_new=[max(i,epsilon) for i in y_pred]
    y_pred_new=[min(i,1-epsilon) for i in y_pred_new]
    y_pred_new=np.array(y_pred_new)
    return -np.mean(y_truth*np.log(y_pred_new)+(1-y_truth)*np.log(1-y_pred_new))

In [135]:
def gradient_descent(x1,x2,y_truth,epochs,loss_threshold):
    w1=w2=1
    b=0
    learning_rate=0.5
    n=len(x1)
    for i in range (epochs):
        weighted_sum=w1*x1+w2*x2+b
        y_pred=1/(1+np.exp(-weighted_sum))
        loss=log_loss(y_truth,y_pred)
        
        w1d=(1/n)*np.dot(np.transpose(x1),(y_pred-y_truth))
        w2d=(1/n)*np.dot(np.transpose(x2),(y_pred-y_truth))
        bd=np.mean(y_pred-y_truth)
        
        w1=w1-learning_rate*w1d
        w2=w2-learning_rate*w2d
        b=b-learning_rate*bd
        
        if loss <= loss_threshold:
            break
            
    return w1,w2,b

In [136]:
gradient_descent(x_train_scaled['age'],x_train_scaled['affordibility'],y_train,367,0.46375862615603475)

(5.00966212989009, 1.4555970892403576, -2.9415522492234514)